#clustering with pca and umap

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
import umap
from tqdm import tqdm

sns.set(style="white") 

# 设置文件路径
data_dir = "./processed_results"  
output_dir = "./clustering"
os.makedirs(output_dir, exist_ok=True)

# =========================================================
# 
# =========================================================
def smiles_to_fp(smiles, nBits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=nBits)
        arr = np.zeros((1,), dtype=int)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr
    else:
        return None

def compute_fingerprints(df):
    fps = []
    valid_indices = []
    for i, smi in enumerate(df['SMILES']):
        fp = smiles_to_fp(smi)
        if fp is not None:
            fps.append(fp)
            valid_indices.append(i)
    return np.array(fps), df.iloc[valid_indices].reset_index(drop=True)

def compute_cluster_metrics(X, labels):
    silhouette = silhouette_score(X, labels)
    centers = np.array([X[labels == i].mean(axis=0) for i in np.unique(labels)])
    dist_mat = pairwise_distances(centers)
    separation = dist_mat[np.triu_indices_from(dist_mat, 1)].mean()
    return silhouette, separation

def get_cluster_centroids(X, labels, fps, smiles):
    centroids = []
    for i in np.unique(labels):
        cluster_idx = np.where(labels == i)[0]
        cluster_vecs = fps[cluster_idx]
        center = cluster_vecs.mean(axis=0)
        dists = np.linalg.norm(cluster_vecs - center, axis=1)
        min_idx = cluster_idx[np.argmin(dists)]
        centroids.append({
            "cluster": i,
            "center_smile": smiles[min_idx],
            "center_index": min_idx
        })
    return pd.DataFrame(centroids)

# =========================================================
# drawing function
# =========================================================
def plot_clusters(X, labels, method_name, k, silhouette, separation, file_prefix, cluster_sizes):
    
    fig, ax = plt.subplots(figsize=(8, 6))  

    palette = sns.color_palette("pastel", len(np.unique(labels)))
    sns.scatterplot(
        x=X[:, 0], y=X[:, 1],
        hue=labels, palette=palette,
        s=50,               
        legend=False, ax=ax
    )

    # 
    for i in np.unique(labels):
        ax.text(
            X[labels == i, 0].mean(),
            X[labels == i, 1].mean(),
            str(cluster_sizes[i]),
            fontsize=17,           
            ha='center', va='center',
            color='black', fontweight='bold'
        )

    ax.set_xlabel(f"{method_name} Dim 1", fontsize=17)   # 
    ax.set_ylabel(f"{method_name} Dim 2", fontsize=17)

    # 
    ax.tick_params(
        direction='out',
        length=8,        
        width=2,         
        color='black',
        bottom=True, left=True, top=False, right=False,
        labelsize=16     # 
    )

    ax.grid(False)
    ax.locator_params(axis='both', nbins=5)

    # feature of title enlarge double size
    plt.figtext(
        0.5, -0.04,
        f"{method_name} | k={k} | silhouette={silhouette:.2f} | separation={separation:.2f}",
        ha='center', fontsize=17  # 
    )

    plt.tight_layout()
    plt.savefig(
        f"{file_prefix}_{method_name}_k{k}.png",
        dpi=600, bbox_inches='tight'
    )
    plt.close()

# =========================================================
# main process
# =========================================================
for file_name in os.listdir(data_dir):
    if file_name.endswith(".csv"):
        file_path = os.path.join(data_dir, file_name)
        df_full = pd.read_csv(file_path)
        smiles_list = df_full["SMILES"].dropna().unique()
        df = pd.DataFrame({"SMILES": smiles_list})

        fps, df_valid = compute_fingerprints(df)

        # PCA & UMAP
        pca_proj = PCA(n_components=2).fit_transform(fps)
        umap_proj = umap.UMAP(n_components=2, random_state=42).fit_transform(fps)

        results = []

        folder_name = file_name.replace('.csv', '').replace('dedup_f-', '').replace('_cleaned', '')
        output_subdir = os.path.join(output_dir, folder_name)
        os.makedirs(output_subdir, exist_ok=True)

        for k in tqdm([2, 3, 4, 5, 6], desc=f"Clustering {file_name}"):
            # PCA clustering
            kmeans = KMeans(n_clusters=k, random_state=42)
            labels_pca = kmeans.fit_predict(pca_proj)
            sil_pca, sep_pca = compute_cluster_metrics(pca_proj, labels_pca)

            # UMAP clustering
            kmeans = KMeans(n_clusters=k, random_state=42)
            labels_umap = kmeans.fit_predict(umap_proj)
            sil_umap, sep_umap = compute_cluster_metrics(umap_proj, labels_umap)

            results.append({
                "k": k,
                "pca_silhouette": sil_pca,
                "pca_separation": sep_pca,
                "umap_silhouette": sil_umap,
                "umap_separation": sep_umap
            })

            prefix = os.path.join(output_subdir, file_name.replace('.csv', ''))

            # map to the whole data
            pca_cluster_map = dict(zip(df_valid['SMILES'], labels_pca))
            umap_cluster_map = dict(zip(df_valid['SMILES'], labels_umap))
            df_full['PCA_Cluster'] = df_full['SMILES'].map(pca_cluster_map).fillna(-1).astype(int)
            df_full['UMAP_Cluster'] = df_full['SMILES'].map(umap_cluster_map).fillna(-1).astype(int)
            df_full.to_csv(f"{prefix}_with_labels_k{k}.csv", index=False)

            # cluster_centroids
            df_centroid_pca = get_cluster_centroids(pca_proj, labels_pca, fps, df_valid["SMILES"].values)
            df_centroid_umap = get_cluster_centroids(umap_proj, labels_umap, fps, df_valid["SMILES"].values)
            df_centroid_pca.to_csv(f"{prefix}_PCA_k{k}_centroids.csv", index=False)
            df_centroid_umap.to_csv(f"{prefix}_UMAP_k{k}_centroids.csv", index=False)

            cluster_sizes_pca = {i: np.sum(labels_pca == i) for i in np.unique(labels_pca)}
            cluster_sizes_umap = {i: np.sum(labels_umap == i) for i in np.unique(labels_umap)}

            #draw
            plot_clusters(pca_proj, labels_pca, "PCA", k, sil_pca, sep_pca, prefix, cluster_sizes_pca)
            plot_clusters(umap_proj, labels_umap, "UMAP", k, sil_umap, sep_umap, prefix, cluster_sizes_umap)

        # save evaluated results
        df_result = pd.DataFrame(results)
        df_result.to_csv(
            os.path.join(output_subdir, f"{file_name.replace('.csv','')}_cluster_evaluation.csv"),
            index=False
        )

[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerator
[15:12:12] DEPRECATION WARNING: please use MorganGenerat